In [1]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr

import numpy as np
import pandas as pd

import networkx as nx

In [2]:
session = WolframLanguageSession("/usr/local/Wolfram/WolframEngine/12.2/Executables/WolframKernel")

In [3]:
with open("../data/datosDcompres.txt","r") as f:
    datosDcompres = f.read()
    
with open("../data/datosTcompres.txt","r") as f:
    datosTcompres = f.read()

In [4]:
session.evaluate('datosDcompres = "{}";'.format(datosDcompres))
session.evaluate('datosTcompres = "{}";'.format(datosTcompres))

In [62]:
def aseverar_valor(variable,nombre_variable):
    try:
        assert session.evaluate(nombre_variable) == variable,"Discrepancia en {}".format(nombre_variable)
    except AssertionError:
        assert (abs(session.evaluate(nombre_variable) - variable) < pow(10,-7)),"Discrepancia en {}".format(nombre_variable)
        

def aseverar_matriz(variable,nombre_variable):
    try:
        assert np.all(np.array(session.evaluate(nombre_variable)) == variable),"Discrepancia en {}".format(nombre_variable)
    except AssertionError:
        assert np.all(np.abs(np.array(session.evaluate(nombre_variable)) - variable) < pow(10,-7)),"Discrepancia en {}".format(nombre_variable)



## Carga de datos

In [6]:
#datosD = Uncompress[  datosDcompres];(*Archivo con datos de la MIP doméstica*)
session.evaluate('datosD = Uncompress[  datosDcompres];')
datosD = pd.read_excel("../data/mip_d_pb_pxp__4_22_42_202_133.xlsx",skiprows=5,index_col=0)

#datosT = Uncompress[  datosTcompres];(*Archivo con datos de la MIP total*)
session.evaluate('datosT = Uncompress[  datosTcompres];')
datosT = pd.read_excel("../data/mip_t_pb_pxp__4_22_45_202_133.xlsx",skiprows=5,index_col=0)

In [7]:
def obtenerSubmatriz(renglones,columnas,datos):
    datos2 = datos.fillna(0)
    datos_interes = datos2.loc[renglones[0]:renglones[1],columnas[0]:columnas[1]]
    matriz = np.array(datos_interes) 
    return matriz

In [8]:
flujos_intermedios = ["111110 - Cultivo de soya","931810 - Actividades de seguridad nacional"]
#Zd = datosD[[8 ;; 829, 4 ;; 825]] /.   "" -> 0.;(*Matriz de flujos intermedios domésticos*)
session.evaluate('Zd = datosD[[8 ;; 829, 4 ;; 825]] /.   "" -> 0.;')
Zd = obtenerSubmatriz(flujos_intermedios,flujos_intermedios,datosD)
    
#Zt = datosT[[8 ;; 829, 4 ;; 825]] /.   "" -> 0.;(*Matriz de flujos intermedios totales*)
session.evaluate('Zt = datosT[[8 ;; 829, 4 ;; 825]] /.   "" -> 0.;')
Zt = obtenerSubmatriz(flujos_intermedios,flujos_intermedios,datosT)

In [9]:
aseverar_matriz(Zd,"Zd")
aseverar_matriz(Zt,"Zt")

In [10]:
demanda_final = ["CP - Consumo Privado","YA0 - Discrepancia estadística"]
# Fd = datosD[[8 ;; 829, 827 ;; 832]] /.   "" -> 0.;(*Matriz de demanda final doméstica*)
session.evaluate('Fd = datosD[[8 ;; 829, 827 ;; 832]] /.   "" -> 0.;')
Fd = obtenerSubmatriz(flujos_intermedios,demanda_final,datosD)
#Ft = datosT[[8 ;; 829, {827, 828, 829, 830, 831, 833}]] /.   "" -> 0.;(*Matriz de demanda final total*)
session.evaluate('Ft = datosT[[8 ;; 829, {827, 828, 829, 830, 831, 833}]] /.   "" -> 0.;')
Ft = obtenerSubmatriz(flujos_intermedios,demanda_final,datosT)
Ft = np.delete(Ft,-2,1)

In [11]:
aseverar_matriz(Fd,"Fd")
aseverar_matriz(Ft,"Ft")

In [12]:
#etiSCIAN =  datosD[[8 ;; 829,   1]];(*Etiquetas de los sectores con código SCIAN*)
session.evaluate('etiSCIAN =  datosD[[8 ;; 829,   1]];')
etiSCIAN = datosD.loc[flujos_intermedios[0]:flujos_intermedios[1]].index
#eti = StringDrop[datosD[[8 ;; 829, 1]],   9];(*Etiquetas de los sectores sin código SCIAN*)
session.evaluate('eti = StringDrop[datosD[[8 ;; 829, 1]],   9];')
eti = etiSCIAN.str.slice(9)

In [13]:
aseverar_matriz(etiSCIAN,"etiSCIAN")
aseverar_matriz(eti,"eti")

In [14]:
#REM = datosD[[844, 4 ;; 825]] /.   "" -> 0.;(*Vector de Remuneraciones a los asalariados*)
session.evaluate('REM = datosD[[844, 4 ;; 825]] /.   "" -> 0.;')
REM = obtenerSubmatriz(["D.1 - Remuneración de los asalariados"]*2,flujos_intermedios,datosD)
#PT = datosD[[855, 4 ;; 825]] /.   "" -> 0.;(*Vector de Puestos de Trabajo totales*)
session.evaluate('PT = datosD[[855, 4 ;; 825]] /.   "" -> 0.;')
PT = obtenerSubmatriz(["PT - Puestos de trabajo"]*2,flujos_intermedios,datosD)
#PTR = datosD[[857, 4 ;; 825]] /.  "" -> 0.;(*Vector de Puestos de Trabajo Remunerados*)
session.evaluate('PTR = datosD[[857, 4 ;; 825]] /.  "" -> 0.;')
PTR = obtenerSubmatriz(["PTR - Puestos de trabajo remunerados"]*2,flujos_intermedios,datosD)
#PIB = datosD[[842, 4 ;; 825]] /.  "" -> 0.;(*Vector de PIB (VAB+impuestos*)
session.evaluate('PIB = datosD[[842, 4 ;; 825]] /.  "" -> 0.;')
PIB = obtenerSubmatriz(["B.1bP - Producto interno bruto"]*2,flujos_intermedios,datosD)

In [15]:
aseverar_matriz(REM,"REM")
aseverar_matriz(PT,"PT")
aseverar_matriz(PTR,"PTR")
aseverar_matriz(PIB,"PIB")

In [90]:
# invD = DiagonalMatrix[Table[If[i == 0, 0, 1/i], {i, #}]] &;
session.evaluate('invD = DiagonalMatrix[Table[If[i == 0, 0, 1/i], {i, #}]] &;')
def invD(vector):
    if vector.ndim > 1 : 
        vector = vector.flatten()
    invertida = np.where(vector==0,0,1/vector)
    diagonal = np.diag(invertida)
    return diagonal

## DEFINICIÓN DE VARIABLES CALCULADAS DE LOS DATOS

In [91]:
#n = Length[Zd];(*Número de actividades en la matriz*)
session.evaluate('n = Length[Zd];')
n = len(Zd)
#fd = Total[Fd\[Transpose]];(*Vector de demanda final doméstica*)
session.evaluate('fd = Total[Fd\[Transpose]];')
fd = np.sum(Fd,1)
#ft = Total[Ft\[Transpose]];(*Vector de demanda final total*)
session.evaluate('ft = Total[Ft\[Transpose]];')
ft = np.sum(Ft,1)
#x = Total[Zd\[Transpose]] +   fd;(*Vector de Valor Bruto de la Producción*)
session.evaluate('x = Total[Zd\[Transpose]] +   fd;')
x = np.sum(Zd,1) + fd
#M = Zt - Zd;(*Matriz de transacciones intermedias de importaciones*)
session.evaluate('M = Zt - Zd;')
M = Zt - Zd
#Fm = Ft - Fd;(*Matriz de demanda final de importaciones*)
session.evaluate('Fm = Ft - Fd;')
Fm = Ft - Fd
#\[Iota] = Table[1, n];(*Vector suma*)
session.evaluate('\[Iota] = Table[1, n];')
Iota = np.ones([1,n])

In [92]:
aseverar_valor(n,"n")
aseverar_matriz(fd,"fd")
aseverar_matriz(ft,"ft")
aseverar_matriz(x,"x")
aseverar_matriz(M,"M")
aseverar_matriz(Fm,"Fm")
aseverar_matriz(Iota,"\[Iota]")

## VECTORES DE COEFICIENTES

In [93]:
# ToDO: Sugiero agregar invD(x) como variable para no reprocesar 
#rem = REM . invD[x];(*Coeficientes de Remuneraciones*)
session.evaluate('rem = REM . invD[x];')
rem = np.matmul(REM,invD(x))
#pt = PT . invD[x];(*Coeficientes de Puestos de Trabajo*)
session.evaluate('pt = PT . invD[x];')
pt = np.matmul(PT,invD(x))
#ptr = PTR . invD[x];(*Coeficientes de Puestos de Trabajo Remunerados*)
session.evaluate('ptr = PTR . invD[x];')
ptr = np.matmul(PTR,invD(x))
#pib = PIB . invD[x];(*Coeficientes de PIB*)
session.evaluate('pib = PIB . invD[x];')
pib = np.matmul(PIB,invD(x))
#imp = Total[M] . invD[x];(*Coeficientes de importaciones (vector)*)
session.evaluate('imp = Total[M] . invD[x];')
imp = np.matmul(np.sum(M,0),invD(x))

In [94]:
aseverar_matriz(rem,"rem")
aseverar_matriz(pt,"pt")
aseverar_matriz(ptr,"ptr")
aseverar_matriz(pib,"pib")
aseverar_matriz(imp,"imp")

## MATRICES DE COEFICIENTES TÉCNICOS Y MATRIZ INVERSA DE LEONTIEF

In [95]:
#A = Zd . invD[x];(*Matriz de coeficientes técnicos domésticos*)
session.evaluate('A = Zd . invD[x];')
A = np.matmul(Zd,invD(x))
#Am = M . invD[x];(*Matriz de coeficientes técnicos de importaciones*)
session.evaluate('Am = M . invD[x];')
Am = np.matmul(M,invD(x))
#At = A + Am;(*Matriz de coeficientes técnicos totales*)
session.evaluate('At = A + Am;')
At = A + Am
#L = Inverse[IdentityMatrix[n] - A];(*Matriz inversa de Leontief*)
session.evaluate('L = Inverse[IdentityMatrix[n] - A];')
L = np.linalg.inv( np.identity(n) - A)

In [96]:
aseverar_matriz(A,"A")
aseverar_matriz(Am,"Am")
aseverar_matriz(At,"At")
aseverar_matriz(L,"L")

## DEFINICIÓN DE VARIABLES MACROECONÓMICAS

In [97]:
#xtot = Total[x];(*Valor Bruto de la Producción total*)
session.evaluate('xtot = Total[x];')
xtot = np.sum(x)
#PIBtot = Total[PIB];(*PIB total*)
session.evaluate('PIBtot = Total[PIB];')
PIBtot = np.sum(PIB)
#REMtot = Total[REM];(*Remuneraciones totales*)
session.evaluate('REMtot = Total[REM];')
REMtot = np.sum(REM)
#PTtot = Total[PT];(*Puestos de trabajo totales*)
session.evaluate('PTtot = Total[PT];')
PTtot = np.sum(PT)
#PTRtot = Total[PTR];(*Puestos de trabajo remunerados totales*)
session.evaluate('PTRtot = Total[PTR];')
PTRtot = np.sum(PTR)
#IMPtot = Total[Total[M]];(*Importaciones totales*)
session.evaluate('IMPtot = Total[Total[M]];')
IMPtot = np.sum(M)
#macro0 = {xtot, PIBtot, REMtot, PTtot, PTRtot, IMPtot};(*Lista de variables macro originales*)
session.evaluate('macro0 = {xtot, PIBtot, REMtot, PTtot, PTRtot, IMPtot};')
macro0 = np.array([xtot, PIBtot, REMtot, PTtot, PTRtot, IMPtot])

In [98]:
aseverar_valor(xtot,"xtot")
aseverar_valor(PIBtot,"PIBtot")
aseverar_valor(REMtot,"REMtot")
aseverar_valor(PTtot,"PTtot")
aseverar_valor(PTRtot,"PTRtot")
aseverar_valor(IMPtot,"IMPtot")
aseverar_matriz(macro0,"macro0")


## DEFINICIÓN DE OTRAS ETIQUETAS 

In [99]:
#etiMacro = {"Producción (millones de pesos)",    "PIB (millones de pesos)", "Remuneraciones (millones de pesos)",    "Puestos de Trabajo", "Puestos de Trabajo Remunerados",    "Importaciones (millones de pesos)"};
session.evaluate('etiMacro = {"Producción (millones de pesos)",    "PIB (millones de pesos)", "Remuneraciones (millones de pesos)",    "Puestos de Trabajo", "Puestos de Trabajo Remunerados",    "Importaciones (millones de pesos)"};')
etiMacro = ["Producción (millones de pesos)",    "PIB (millones de pesos)", "Remuneraciones (millones de pesos)",    "Puestos de Trabajo", "Puestos de Trabajo Remunerados",    "Importaciones (millones de pesos)"]

In [100]:
aseverar_matriz(etiMacro,"etiMacro")

## FUNCIÓN PARA HACER LAS SIMULACIONES CUANTITATIVAS

In [27]:
session.evaluate('''
simulacion[lista_] := Module[
  (*Lista de variables locales*)
  {numSust, listaSustA, listaSustAm, Amod, AMmod, Lmod, xmod, PIBmod, 
   VABmod, REMmod, PTmod, PTRmod, IMPmod, PIBmodVec, REMmodVec, 
   PTmodVec, PTRmodVec, tablaMacro, resMacro, tablaPIB, tablaREM, 
   tablaPTR, tablaPT},
  
  (*-- Preparación de variables de apoyo --*)
  numSust = Length[lista];(*Número de insumos a integrar*)
  (*Reglas de sustitución para la matriz de coeficientes técnicos*)
  listaSustA = Table[
    {lista[[i, 1]], 
      lista[[i, 
       2]]} -> (A[[lista[[i, 1]], 
        lista[[i, 2]]]] + (lista[[i, 3]]/100)*
        Am[[lista[[i, 1]], lista[[i, 2]]]]),
    {i, numSust}
    ];
  (*Reglas de sustitución para la matriz de coeficientes importados*)

  
  listaSustAm = Table[
    {lista[[i, 1]], 
      lista[[i, 2]]} -> ((1 - lista[[i, 3]]/100)*
       Am[[lista[[i, 1]], lista[[i, 2]]]]),
    {i, numSust}
    ];
  (*- Fin de preparación de variables de apoyo -*)
  
  (*-- Estimación de la simulación --*)
  Amod = ReplacePart[A, 
    listaSustA];(*Matriz de coeficientes técnicos modificada*)
  AMmod = 
   ReplacePart[Am, 
    listaSustAm];(*Matriz de coeficientes importados modificada*)
  Lmod = Inverse[
    IdentityMatrix[n] - 
     Amod];(*Inversa de Leontief doméstica modificada*)
  xmod = Lmod . fd;(*Vector de VBP modificado*)
  PIBmod = pib . xmod;(*Total del PIB modificado*)
  PIBmodVec = pib*xmod;(*Vector del PIB por actividad modificado*)
  REMmod = rem . xmod;(*Total de Remuneraciones modificadas*)
  REMmodVec = 
   rem*xmod;(*Vector de Remuneraciones por actividad modificadas*)
  PTmod = pt . xmod;(*Total del PT modificados*)
  PTmodVec = pt*xmod;(*Vector de PT por actividad modificados*)
  PTRmod = ptr . xmod;(*Total del PTR modificados*)
  PTRmodVec = ptr*xmod;(*Vector del PTR por actividad modificados*)
  IMPmod = 
   Total[AMmod] . xmod;(*Total del importaciones modificadas*)
  (*- Fin de estimación de la simulación -*)
  
  (*-- Generación de tablas con resultados*)
  
  (*Tabla de resultados macro*)
  resMacro = {Total[xmod] - xtot, PIBmod - PIBtot, REMmod - REMtot, 
    PTmod - PTtot, PTRmod - PTRtot, IMPmod - IMPtot};
  tablaMacro = AccountingForm[
    TableForm[{etiMacro, macro0, 
       Round[resMacro, 
        0.001], (Round[resMacro, 0.001]/macro0*100)}\[Transpose],
     TableHeadings -> {None, {"Variable", 
        "Niveles originales \n(millones de pesos)", 
        "Variación \n(millones de pesos)", 
        "Variación porcentual \n(%)"}}]
    ,
    DigitBlock -> 3,
    NumberSigns -> {"-", ""},
    NumberSeparator -> {",", ""}
    ];
  (*Tabla de principales sectores afectados vía PIB*)
  tablaPIB = AccountingForm[
    TableForm[
     Reverse[
       SortBy[{etiSCIAN, PIB, Round[PIBmodVec - PIB, 0.001], 
          Round[(PIBmodVec . invD[PIB] - 1)*100, 0.001]}\[Transpose],
        #[[3]] &
        ]][[1 ;; 20]]
     ,
     TableHeadings -> {None, {"Variable", 
        "Niveles originales \n(millones de pesos)", 
        "Variación \n(millones de pesos)", 
        "Variación porcentual \n(%)"}}]
    ,
    DigitBlock -> 3,
    NumberSigns -> {"-", ""},
    NumberSeparator -> {",", ""}
    ];
  (*Tabla de principales sectores afectados vía Remuneraciones*)
  tablaREM = AccountingForm[
    TableForm[
     Reverse[
       SortBy[{etiSCIAN, REM, Round[REMmodVec - REM, 0.001], 
          Round[(REMmodVec . invD[REM] - 1)*100, 0.001]}\[Transpose],
        #[[3]] &
        ]][[1 ;; 20]]
     ,
     TableHeadings -> {None, {"Variable", 
        "Niveles originales \n(millones de pesos)", 
        "Variación \n(millones de pesos)", 
        "Variación porcentual \n(%)"}}]
    ,
    DigitBlock -> 3,
    NumberSigns -> {"-", ""},
    NumberSeparator -> {",", ""}
    ];
  (*Tabla de principales sectores afectados vía Puestos de Trabajo \
Remunerados*)
  tablaPTR = AccountingForm[
    TableForm[
     Reverse[
       SortBy[{etiSCIAN, PTR, Round[PTRmodVec - PTR, 0.001], 
          Round[(PTRmodVec . invD[PTR] - 1)*100, 0.001]}\[Transpose],
        #[[3]] &
        ]][[1 ;; 20]]
     ,
     TableHeadings -> {None, {"Variable", 
        "Niveles originales \n(Puestos)", "Variación \n(Puestos)", 
        "Variación porcentual \n(%)"}}]
    ,
    DigitBlock -> 3,
    NumberSigns -> {"-", ""},
    NumberSeparator -> {",", ""}
    ];
  (*Tabla de principales sectores afectados vía Puestos de Trabajo*)
  tablaPT = AccountingForm[
    TableForm[
     Reverse[SortBy[
        {etiSCIAN, PT, Round[PTmodVec - PT, 0.001], 
          Round[(PTmodVec . invD[PT] - 1)*100, 0.001]}\[Transpose],
        #[[3]] &
        ]][[1 ;; 20]]
     ,
     TableHeadings -> {None, {"Variable", 
        "Niveles originales \n(Puestos)", "Variación \n(Puestos)", 
        "Variación porcentual \n(%)"}}]
    ,
    DigitBlock -> 3,
    NumberSigns -> {"-", ""},
    NumberSeparator -> {",", ""}
    ];
  
  (*Reporte de resultados*)
  {tablaMacro, tablaPIB, tablaREM, tablaPTR, tablaPT}
  ]
''')

In [102]:
#def simulacion(lista):
lista = [[272,93,100]]
#(*-- Preparación de variables de apoyo --*)
#numSust = Length[lista];(*Número de insumos a integrar*)
session.evaluate('lista = {{273,94,100}}')
session.evaluate('numSust = Length[lista];')
numSust = len(lista)
aseverar_valor(numSust,"numSust")
#(*Reglas de sustitución para la matriz de coeficientes técnicos*)
#listaSustA = Table[
#{lista[[i, 1]], lista[[i, 2]]} -> (A[[lista[[i, 1]], lista[[i, 2]]]] + (lista[[i, 3]]/100)* Am[[lista[[i, 1]], lista[[i, 2]]]]),
#{i, numSust}
#];
#(*Reglas de sustitución para la matriz de coeficientes importados*)
#listaSustAm = Table[
#{lista[[i, 1]], lista[[i, 2]]} -> ((1 - lista[[i, 3]]/100)* Am[[lista[[i, 1]], lista[[i, 2]]]]),
# {i, numSust}
#];
session.evaluate('''listaSustA = Table[
{lista[[i, 1]], 
lista[[i, 
2]]} -> (A[[lista[[i, 1]], 
lista[[i, 2]]]] + (lista[[i, 3]]/100)*
Am[[lista[[i, 1]], lista[[i, 2]]]]),
{i, numSust}
];
listaSustAm = Table[
{lista[[i, 1]], 
lista[[i, 2]]} -> ((1 - lista[[i, 3]]/100)*
Am[[lista[[i, 1]], lista[[i, 2]]]]),
{i, numSust}
];
''')
#(*- Fin de preparación de variables de apoyo -*)
#(*-- Estimación de la simulación --*)
#Amod = ReplacePart[A, listaSustA];(*Matriz de coeficientes técnicos modificada*)
#AMmod = ReplacePart[Am, listaSustAm];(*Matriz de coeficientes importados modificada*)
session.evaluate('Amod = ReplacePart[A, listaSustA];')
session.evaluate('AMmod = ReplacePart[Am, listaSustAm];')
Amod = A.copy()
AMmod = Am.copy()
for elem in lista:
    Amod[elem[0],elem[1]] += (elem[2]/100)*Am[elem[0],elem[1]]
    AMmod[elem[0],elem[1]] = (1 - elem[2]/100)* Am[elem[0],elem[1]]

aseverar_matriz(Amod,"Amod")
aseverar_matriz(AMmod,"AMmod")

#Lmod = Inverse[ IdentityMatrix[n] -  Amod];(*Inversa de Leontief doméstica modificada*)
session.evaluate('Lmod = Inverse[ IdentityMatrix[n] -  Amod];')
Lmod = np.linalg.inv( np.identity(n) - Amod)
#xmod = Lmod . fd;(*Vector de VBP modificado*)
session.evaluate('xmod = Lmod . fd;')
xmod = np.matmul(Lmod , fd)
#PIBmod = pib . xmod;(*Total del PIB modificado*)
session.evaluate('PIBmod = pib . xmod;')
PIBmod = np.matmul(pib,  xmod)[0]
#PIBmodVec = pib*xmod;(*Vector del PIB por actividad modificado*)
session.evaluate('PIBmodVec = pib*xmod;')
PIBmodVec = pib*xmod
#REMmod = rem . xmod;(*Total de Remuneraciones modificadas*)
session.evaluate('REMmod = rem . xmod;')
REMmod = np.matmul(rem, xmod)[0]
#REMmodVec = rem*xmod;(*Vector de Remuneraciones por actividad modificadas*)
session.evaluate('REMmodVec = rem*xmod;')
REMmodVec = rem*xmod
#PTmod = pt . xmod;(*Total del PT modificados*)
session.evaluate('PTmod = pt . xmod;')
PTmod = np.matmul(pt , xmod)[0]
#PTmodVec = pt*xmod;(*Vector de PT por actividad modificados*)
session.evaluate('PTmodVec = pt*xmod;')
PTmodVec = pt*xmod
#PTRmod = ptr . xmod;(*Total del PTR modificados*)
session.evaluate('PTRmod = ptr . xmod;')
PTRmod = np.matmul(ptr , xmod)[0]
#PTRmodVec = ptr*xmod;(*Vector del PTR por actividad modificados*)
session.evaluate('PTRmodVec = ptr*xmod;')
PTRmodVec = ptr*xmod
#IMPmod = Total[AMmod] . xmod;(*Total del importaciones modificadas*)
session.evaluate('IMPmod = Total[AMmod] . xmod;')
IMPmod = np.matmul( np.sum(AMmod,0), xmod)

aseverar_matriz(Lmod,"Lmod")
aseverar_matriz(xmod,"xmod")
aseverar_matriz(PIBmod,"PIBmod")
aseverar_matriz(PIBmodVec,"PIBmodVec")
aseverar_matriz(REMmod,"REMmod")
aseverar_matriz(REMmodVec,"REMmodVec")
aseverar_matriz(PTmod,"PTmod")
aseverar_matriz(PTmodVec,"PTmodVec")
aseverar_matriz(PTRmod,"PTRmod")
aseverar_matriz(PTRmodVec,"PTRmodVec")
aseverar_matriz(IMPmod,"IMPmod")

#(*- Fin de estimación de la simulación -*)

#(*-- Generación de tablas con resultados*)
#(*Tabla de resultados macro*)
#resMacro = {Total[xmod] - xtot, PIBmod - PIBtot, REMmod - REMtot, PTmod - PTtot, PTRmod - PTRtot, IMPmod - IMPtot};
resMacro = [np.sum(xmod) - xtot, PIBmod - PIBtot, REMmod - REMtot, PTmod - PTtot, PTRmod - PTRtot, IMPmod - IMPtot]
#tablaMacro = AccountingForm[
#TableForm[{etiMacro, macro0, Round[resMacro, 0.001], (Round[resMacro, 0.001]/macro0*100)}\[Transpose],
# TableHeadings -> {None, {"Variable", "Niveles originales \n(millones de pesos)", "Variación \n(millones de pesos)", "Variación porcentual \n(%)"}}]    ,
# DigitBlock -> 3,
# NumberSigns -> {"-", ""},
# NumberSeparator -> {",", ""}
#];
#tablaMacro = pd.DataFrame([etiMacro,macro0,np.round(resMacro,3),np.round(resMacro, 0.001)/macro0*100], columns = ["Variable", "Niveles originales (millones de pesos)", "Variación (millones de pesos)", "Variación porcentual (%)"])
#(*Tabla de principales sectores afectados vía PIB*)
#tablaPIB = AccountingForm[
#TableForm[
# Reverse[SortBy[{etiSCIAN, PIB, Round[PIBmodVec - PIB, 0.001], Round[(PIBmodVec . invD[PIB] - 1)*100, 0.001]}\[Transpose],#[[3]] &]][[1 ;; 20]]     ,
# TableHeadings -> {None, {"Variable", "Niveles originales \n(millones de pesos)", "Variación \n(millones de pesos)", "Variación porcentual \n(%)"}}] ,
# DigitBlock -> 3,
# NumberSigns -> {"-", ""},
# NumberSeparator -> {",", ""}
#];
#tablaPIB = pd.DataFrame([etiSCIAN, PIB, np.round(PIBmodVec - PIB, 3), np.round((PIBmodVec . invD(PIB) - 1)*100, 3)], columns = ["Variable", "Niveles originales (millones de pesos)", "Variación (millones de pesos)", "Variación porcentual (%)"])

#(*Tabla de principales sectores afectados vía Remuneraciones*)
#tablaREM = AccountingForm[
#TableForm[
# Reverse[SortBy[{etiSCIAN, REM, Round[REMmodVec - REM, 0.001], Round[(REMmodVec . invD[REM] - 1)*100, 0.001]}\[Transpose], #[[3]] & ]][[1 ;; 20]] ,
# TableHeadings -> {None, {"Variable", "Niveles originales \n(millones de pesos)", "Variación \n(millones de pesos)", "Variación porcentual \n(%)"}}] ,
# DigitBlock -> 3,
# NumberSigns -> {"-", ""},
# NumberSeparator -> {",", ""}
#];
#(*Tabla de principales sectores afectados vía Puestos de Trabajo Remunerados*)
#tablaPTR = AccountingForm[
#TableForm[
# Reverse[SortBy[{etiSCIAN, PTR, Round[PTRmodVec - PTR, 0.001], Round[(PTRmodVec . invD[PTR] - 1)*100, 0.001]}\[Transpose],#[[3]] & ]][[1 ;; 20]]  ,
# TableHeadings -> {None, {"Variable", "Niveles originales \n(Puestos)", "Variación \n(Puestos)",  "Variación porcentual \n(%)"}}] ,
# DigitBlock -> 3,
# NumberSigns -> {"-", ""},
# NumberSeparator -> {",", ""}
#];
#(*Tabla de principales sectores afectados vía Puestos de Trabajo*)
#tablaPT = AccountingForm[
#TableForm[ Reverse[SortBy[ {etiSCIAN, PT, Round[PTmodVec - PT, 0.001],  Round[(PTmodVec . invD[PT] - 1)*100, 0.001]}\[Transpose], #[[3]] & ]][[1 ;; 20]] ,
# TableHeadings -> {None, {"Variable",  "Niveles originales \n(Puestos)", "Variación \n(Puestos)",  "Variación porcentual \n(%)"}}] ,
# DigitBlock -> 3,
# NumberSigns -> {"-", ""},
# NumberSeparator -> {",", ""}
#];

#(*Reporte de resultados*)
#{tablaMacro, tablaPIB, tablaREM, tablaPTR, tablaPT}
   # return [tablaMacro, tablaPIB]
    #]

In [ ]:
#eti2 = StringReplace[etiSCIAN, {
#   "Fabricación de" -> "Fab.",
#   "Cultivo de" -> "C.",
#   "Explotación de" -> "Exp.",
#   "Producción" -> "Prod.",
#   "producción" -> "prod.",
#   "Minería de" -> "Min.",
#   "Elaboración de" -> "Elab.",
#   "Transporte de" -> "Trans.",
#   "Transporte" -> "Trans.",
#   "Alquiler" -> "Alq.",
#   "Servicios de" -> "S.",
#   "Servicios" -> "S."
#   }];(*Sustitución de algunas palabras en las etiquetas por \
#abreviaciones*)
session.evaluate('''
eti2 = StringReplace[etiSCIAN, {
   "Fabricación de" -> "Fab.",
   "Cultivo de" -> "C.",
   "Explotación de" -> "Exp.",
   "Producción" -> "Prod.",
   "producción" -> "prod.",
   "Minería de" -> "Min.",
   "Elaboración de" -> "Elab.",
   "Transporte de" -> "Trans.",
   "Transporte" -> "Trans.",
   "Alquiler" -> "Alq.",
   "Servicios de" -> "S.",
   "Servicios" -> "S."
   }];
''')

reemp_dic = {
   "Fabricación de" : "Fab.",
   "Cultivo de" : "C.",
   "Explotación de" : "Exp.",
   "Producción" : "Prod.",
   "producción" : "prod.",
   "Minería de" : "Min.",
   "Elaboración de" : "Elab.",
   "Transporte de" : "Trans.",
   "Transporte" : "Trans.",
   "Alquiler" : "Alq.",
   "Servicios de" : "S.",
   "Servicios" : "S."
   }
eti2 = list(etiSCIAN)
for key,val in reemp_dic.items():
    eti2 = list(map(lambda s: s.replace(key, val), eti2))
    
aseverar_matriz(eti2,"eti2") 


In [ ]:
session.evaluate('''
PrepararRed[filtro_] := Module[{Bd, Bt},
  (*Binarización de matrices según el filtro*)
  Bd = Table[If[A[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
  Bt = Table[If[At[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
  (*Eliminación de relaciones hacia adelante del comercio*)
  Bd = ReplacePart[Bd, 
    Flatten[Table[{i, j} -> 0, {i, {447, 448}}, {j, n}], 1]];
  Bt = ReplacePart[Bt, 
    Flatten[Table[{i, j} -> 0, {i, {447, 448}}, {j, n}], 1]];
  (*Eliminación de auto-loops*)
  Bd = ReplacePart[Bd, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
  Bt = ReplacePart[Bt, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
  (*Generación de las redes*)
  gd = AdjacencyGraph[Bd];
  gt = AdjacencyGraph[Bt];
  (*Redes en forma de lista*)
  RdAtras = Table[Flatten[Position[Bd[[All, i]], 1]], {i, n}];
  RdAdelante = Table[Flatten[Position[Bd[[i]], 1]], {i, n}];
  RtAtras = Table[Flatten[Position[Bt[[All, i]], 1]], {i, n}];
  RtAdelante = Table[Flatten[Position[Bt[[i]], 1]], {i, n}];
  ]
''')

def PrepararRed (filtro):
    #PrepararRed[filtro_] := Module[{Bd, Bt},
    #(*Binarización de matrices según el filtro*)
    # Bd = Table[If[A[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
    # Bt = Table[If[At[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
    session.evaluate('''
      Bd = Table[If[A[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
      Bt = Table[If[At[[i, j]] >= filtro, 1, 0], {i, n}, {j, n}];
    ''')
    Bd = np.where(A>filtro,1,0)
    Bt = np.where(At>filtro,1,0)
    return [Bd,Bt]
    #aseverar_matriz(Bd,"Bd") 
    #aseverar_matriz(Bt,"Bt") 
    
    #(*Eliminación de relaciones hacia adelante del comercio*)
    #Bd = ReplacePart[Bd, Flatten[Table[{i, j} -> 0, {i, {447, 448}}, {j, n}], 1]];
    #Bt = ReplacePart[Bt,     Flatten[Table[{i, j} -> 0, {i, {447, 448}}, {j, n}], 1]];
        
    
    Bd[:,[447, 448]] = 0
    Bt[:,[447, 448]] = 0
    
    
    #aseverar_matriz(Bd,"Bd") 
    #aseverar_matriz(Bt,"Bt") 
    
    #(*Eliminación de auto-loops*)
    #Bd = ReplacePart[Bd, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
    #Bt = ReplacePart[Bt, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
    session.evaluate('''
      Bd = ReplacePart[Bd, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
      Bt = ReplacePart[Bt, Flatten[Table[{i, i} -> 0, {i, n}], 1]];
    ''')
    #Bd -= np.diag(Bd)
    #Bt -= np.diag(Bt)
    
    #aseverar_matriz(Bd,"Bd") 
    #aseverar_matriz(Bt,"Bt") 
    return [Bd,Bt]
    

In [ ]:
Bd,Bt = PrepararRed(0.04)

In [ ]:
DG = nx.DiGraph()
DG.add_weighted_edges_from([(etiSCIAN[i],etiSCIAN[j],destino) for i, origen in enumerate(At) for j, destino in enumerate(origen) if destino > 0.04 ])
    
        #if destino > 0.04:
            

In [ ]:
nx.json_graph.node_link_data(nx.bfs_tree(DG,etiSCIAN[0],True))

## DEFINICIÓN DE LAS FUNCIONES DE PSEUDOCLAUSURA

In [ ]:
#(*Función de adherencia hacia atrás de relaciones domésticas*)
#a1[V_ /; Length[V] == 0] := RdAtras[[V]] \[Union] {V}
#a1[V_ /; Length[V] != 0] := Union @@ (RdAtras[[V]]) \[Union] V
#(*Función de adherencia hacia adelante de relaciones domésticas*)
#a2[V_ /; Length[V] == 0] := RdAdelante[[V]] \[Union] {V}
#a2[V_ /; Length[V] != 0] := Union @@ (RdAdelante[[V]]) \[Union] V
#(*Función de adherencia hacia atrás de relaciones totales*)
#a3[V_ /; Length[V] == 0] := RtAtras[[V]] \[Union] {V}
#a3[V_ /; Length[V] != 0] := Union @@ (RtAtras[[V]]) \[Union] V
#(*Función de adherencia hacia adelante de relaciones totales*)
#a4[V_ /; Length[V] == 0] := RtAdelante[[V]] \[Union] {V}
#a4[V_ /; Length[V] != 0] := Union @@ (RtAdelante[[V]]) \[Union] V

## FUNCIÓN PARA ENCONTRAR CONJUNTOS CERRADOS

In [ ]:
#cerrado[sec_, func_] := Module[{F},
#  F = func[sec];
#  While[func[F] != F, F = func[F]];
#  F]

In [ ]:
def CreaRed (inicio,adyacencia,atras = True,filtro=0.04):
    DG = nx.DiGraph()
    DG.add_edges_from([(etiSCIAN[i],etiSCIAN[j],{"cantidad":destino}) 
                                for i, origen in enumerate(adyacencia) 
                                for j, destino in enumerate(origen) 
                                if destino > filtro and i != j and not i in [446, 447]])
    try:
        SDG = DG.subgraph(nx.bfs_tree(DG,inicio,atras).nodes)
    except nx.NetworkXError:
        SDG = nx.DiGraph()
        SDG.add_node(inicio)
    return SDG

nx.json_graph.node_link_data(CreaRed(etiSCIAN[0],At))

In [ ]:
etiSCIAN.get_loc('212299 - Minería de otros minerales metálicos')

## FUNCIÓN PARA GRAFICAR LAS REDES SIN MODIFICACIONES
## FUNCIÓN PARA GRAFICAR LAS REDES CON MODIFICACIONES
## VARIABLES DE APOYO PARA LA INTERFAZ
## SE BORRAN ALGUNAS VARIABLES ANTES DE DESPLEGAR LA INTERFAZ
## DESPLIEGUE DE LA INTERFAZ
Estos elememntos es parte de la visualización por lo que no se implementa aquí 